In [ ]:
# Тренував також
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased' і model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased'

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer
import torch

df = pd.read_csv('new_data_task3/processed_reviews.csv')

df['review_text'] = df['review_text'].str.lower().str.replace(r'[^\w\s]+', '', regex=True)

train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

def tokenize_and_encode(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=128, return_tensors='pt')

train_encodings = tokenize_and_encode(train_df['review_text'].tolist())
test_encodings = tokenize_and_encode(test_df['review_text'].tolist())

class ReviewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(int(self.labels[idx]) - 1)  
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ReviewsDataset(train_encodings, train_df['review_score'])
test_dataset = ReviewsDataset(test_encodings, test_df['review_score'])

c:\Users\Микола\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
from transformers import Trainer, TrainingArguments, RobertaForSequenceClassification
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=5)
training_args = TrainingArguments(
    output_dir='new_data_task3/model_training_results',
    num_train_epochs=15,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    learning_rate=5e-5,
    logging_steps=250,
    evaluation_strategy='steps',  
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/18345 [00:00<?, ?it/s]

{'loss': 1.1555, 'grad_norm': 10.642463684082031, 'learning_rate': 2.5e-05, 'epoch': 0.2}


  0%|          | 0/272 [00:00<?, ?it/s]

c:\Users\Микола\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

  0%|          | 0/272 [00:00<?, ?it/s]

{'eval_loss': 1.2351536750793457, 'eval_accuracy': 0.6731078904991948, 'eval_precision': 0.6515682991408055, 'eval_recall': 0.6731078904991948, 'eval_f1': 0.6595323372835603, 'eval_runtime': 16.6223, 'eval_samples_per_second': 261.516, 'eval_steps_per_second': 16.364, 'epoch': 14.72}
{'loss': 0.3907, 'grad_norm': 12.692825317382812, 'learning_rate': 2.6618100308209587e-07, 'epoch': 14.92}


  0%|          | 0/272 [00:00<?, ?it/s]

{'eval_loss': 1.242987036705017, 'eval_accuracy': 0.6728778467908902, 'eval_precision': 0.6501326112687558, 'eval_recall': 0.6728778467908902, 'eval_f1': 0.6589296147450722, 'eval_runtime': 16.9984, 'eval_samples_per_second': 255.73, 'eval_steps_per_second': 16.002, 'epoch': 14.92}
{'train_runtime': 8061.8703, 'train_samples_per_second': 72.78, 'train_steps_per_second': 2.276, 'train_loss': 0.6337880610509932, 'epoch': 15.0}


TrainOutput(global_step=18345, training_loss=0.6337880610509932, metrics={'train_runtime': 8061.8703, 'train_samples_per_second': 72.78, 'train_steps_per_second': 2.276, 'total_flos': 3.859548472733184e+16, 'train_loss': 0.6337880610509932, 'epoch': 15.0})